# Arvak + Cirq Integration

This notebook demonstrates seamless integration between Arvak and Cirq, enabling:
- Bi-directional circuit conversion (Cirq ↔ Arvak)
- Using Arvak compilation with Cirq circuits
- Leveraging Cirq's hardware-native approaches
- Support for LineQubit and GridQubit layouts

Cirq is Google's quantum computing framework that emphasizes:
- **Hardware-Native**: Circuits designed for specific hardware
- **GridQubit**: 2D qubit layouts for superconducting qubits
- **LineQubit**: 1D qubit arrangements
- **NISQ Algorithms**: QAOA, VQE, and variational circuits
- **Noise Modeling**: Realistic hardware simulation

## Installation

```bash
pip install arvak[cirq]
```

## Step 1: Check Integration Status

First, let's verify that the Cirq integration is available.

In [ ]:
import arvak

# Verify Cirq integration is available
status = arvak.integration_status()
print("Available integrations:")
for name, info in status.items():
    icon = "✓" if info['available'] else "✗"
    print(f"  {icon} {name}: {info['packages']}")

# Check specifically for Cirq
if 'cirq' not in status or not status['cirq']['available']:
    raise ImportError(
        "Cirq integration not available. "
        "Install with: pip install cirq>=1.0.0 cirq-core>=1.0.0"
    )

print("\n✓ Cirq integration is available!")

## Step 2: Create Circuit with LineQubit

Cirq uses LineQubit for 1D qubit arrangements.

In [ ]:
import cirq

# Create qubits using LineQubit (1D arrangement)
qubits = cirq.LineQubit.range(2)

# Create a Bell state circuit in Cirq
circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.measure(*qubits, key='result')
)

print("Cirq Circuit with LineQubit:")
print(circuit)

## Step 3: Convert Cirq Circuit to Arvak

In [ ]:
# Get Cirq integration
cirq_integration = arvak.get_integration('cirq')

# Convert to Arvak
arvak_circuit = cirq_integration.to_arvak(circuit)

print("Arvak Circuit:")
print(f"  Name: {arvak_circuit.name}")
print(f"  Qubits: {arvak_circuit.num_qubits}")
print(f"  Classical bits: {arvak_circuit.num_clbits}")
print(f"  Depth: {arvak_circuit.depth()}")
print(f"  Gate count: {arvak_circuit.size()}")

## Step 4: Create Circuit with GridQubit

GridQubit is used for 2D qubit layouts, common in superconducting quantum processors.

In [ ]:
# Create qubits using GridQubit (2D grid)
# This represents a 2x2 grid
q00 = cirq.GridQubit(0, 0)
q01 = cirq.GridQubit(0, 1)
q10 = cirq.GridQubit(1, 0)
q11 = cirq.GridQubit(1, 1)

# Create a circuit with GridQubits
grid_circuit = cirq.Circuit(
    cirq.H(q00),
    cirq.CNOT(q00, q01),
    cirq.CNOT(q01, q11),
    cirq.CNOT(q00, q10),
    cirq.measure(q00, q01, q10, q11, key='result')
)

print("Cirq Circuit with GridQubit:")
print(grid_circuit)
print(f"\nQubits: {grid_circuit.all_qubits()}")

## Step 5: Convert GridQubit Circuit to Arvak

In [ ]:
# Convert GridQubit circuit to Arvak
arvak_grid = cirq_integration.to_arvak(grid_circuit)

print("Arvak Circuit from GridQubit:")
print(f"  Qubits: {arvak_grid.num_qubits}")
print(f"  Depth: {arvak_grid.depth()}")
print("\nOpenQASM representation:")
print(arvak.to_qasm(arvak_grid))

## Step 6: Convert Arvak Circuit Back to Cirq

In [ ]:
# Create a circuit in Arvak
arvak_original = arvak.Circuit.ghz(3)

# Convert to Cirq
cirq_circuit = cirq_integration.from_arvak(arvak_original)

print("Converted to Cirq:")
print(cirq_circuit)

## Step 7: Use Arvak as Cirq Sampler

Execute Cirq circuits through Arvak's backend using Cirq's Sampler API.

**Note**: Backend execution is currently a mock implementation. For actual execution, use the Arvak CLI:
```bash
arvak run circuit.qasm --backend sim --shots 1000
```

In [ ]:
from arvak.integrations.cirq import ArvakSampler

# Get Arvak sampler for Cirq
sampler = ArvakSampler('sim')
print(f"Sampler: {sampler.name}")

# Run circuit using Cirq's standard API
result = sampler.run(circuit, repetitions=1000)

print("\nExecution Results:")
histogram = result.histogram(key='result')
for outcome, count in sorted(histogram.items()):
    bitstring = format(outcome, f'0{len(qubits)}b')
    bar = "█" * int(count / 10)
    print(f"  {bitstring}: {count:4d} {bar}")

## Step 8: Configure Arvak Compilation for Hardware

In [ ]:
from arvak import CouplingMap, BasisGates, PropertySet, Layout

# Configure for IQM hardware
coupling_map = CouplingMap.star(5)
basis_gates = BasisGates.iqm()
layout = Layout.trivial(5)

props = PropertySet().with_target(coupling_map, basis_gates)
props.set_layout(layout)

print("IQM Hardware Configuration:")
print(f"  Topology: star with 5 qubits")
print(f"  Edges: {coupling_map.edges()}")
print(f"  Native gates: {basis_gates.gates()}")

## Step 9: Cirq's Moment Structure

Cirq organizes operations into "moments" - sets of operations that can execute simultaneously.

In [ ]:
# Create a circuit with explicit moments
q0, q1, q2 = cirq.LineQubit.range(3)

moment_circuit = cirq.Circuit(
    # Moment 0: Apply H gates in parallel
    cirq.Moment([cirq.H(q0), cirq.H(q1), cirq.H(q2)]),
    # Moment 1: Apply CNOT gates
    cirq.Moment([cirq.CNOT(q0, q1)]),
    # Moment 2: Another CNOT
    cirq.Moment([cirq.CNOT(q1, q2)]),
    # Moment 3: Measurements
    cirq.Moment([cirq.measure(q0, q1, q2, key='result')])
)

print("Cirq Circuit with Explicit Moments:")
print(moment_circuit)
print(f"\nNumber of moments: {len(moment_circuit)}")

# Convert to Arvak
arvak_from_moments = cirq_integration.to_arvak(moment_circuit)
print(f"Arvak circuit depth: {arvak_from_moments.depth()}")

## Step 10: Parametrized Circuits

Cirq supports parametrized circuits for variational algorithms.

In [ ]:
import sympy

# Create symbolic parameters
theta = sympy.Symbol('theta')
phi = sympy.Symbol('phi')

# Create parametrized circuit
q0, q1 = cirq.LineQubit.range(2)
param_circuit = cirq.Circuit(
    cirq.rx(theta)(q0),
    cirq.ry(phi)(q1),
    cirq.CNOT(q0, q1),
    cirq.measure(q0, q1, key='result')
)

print("Parametrized Cirq Circuit:")
print(param_circuit)
print(f"\nParameters: {cirq.protocols.parameter_names(param_circuit)}")

# Resolve parameters
import numpy as np
resolved = cirq.resolve_parameters(param_circuit, {
    'theta': np.pi / 4,
    'phi': np.pi / 2
})

print("\nResolved Circuit:")
print(resolved)

# Convert to Arvak
arvak_resolved = cirq_integration.to_arvak(resolved)
print(f"\nArvak circuit: {arvak_resolved.num_qubits} qubits, depth {arvak_resolved.depth()}")

## Step 11: Native Gate Sets

Cirq emphasizes hardware-native gate sets.

In [ ]:
# Google's Sycamore gate (two-qubit gate)
q0, q1 = cirq.LineQubit.range(2)

# Circuit using native gates
native_circuit = cirq.Circuit(
    cirq.X(q0)**0.5,  # sqrt(X) gate
    cirq.Y(q1)**0.5,  # sqrt(Y) gate
    cirq.FSimGate(theta=np.pi/4, phi=0)(q0, q1),  # Fermionic simulation gate
    cirq.measure(q0, q1, key='result')
)

print("Cirq Circuit with Native Gates:")
print(native_circuit)

# Convert to Arvak
arvak_native = cirq_integration.to_arvak(native_circuit)
print(f"\nConverted to Arvak: {arvak_native.num_qubits} qubits")

## Step 12: Export for CLI Execution

In [ ]:
# Convert Cirq circuit to Arvak format
arvak_export = cirq_integration.to_arvak(circuit)

# Export to QASM for CLI execution
output_qasm = arvak.to_qasm(arvak_export)

# Save to file
with open("cirq_circuit.qasm", "w") as f:
    f.write(output_qasm)

print("Circuit exported to cirq_circuit.qasm")
print("\nTo execute:")
print("  $ arvak run cirq_circuit.qasm --backend sim --shots 1000")
print("\nWith specific backend:")
print("  $ arvak run cirq_circuit.qasm --backend iqm --shots 1000")

## Summary

This notebook demonstrated:

1. **Integration Status**: Checking if Cirq integration is available
2. **LineQubit**: 1D qubit arrangements
3. **Conversion to Arvak**: Converting Cirq circuits to Arvak format
4. **GridQubit**: 2D qubit layouts for superconducting chips
5. **Round-Trip Conversion**: Arvak ↔ Cirq circuit conversion
6. **Sampler Interface**: Running circuits through Arvak backend
7. **Hardware Configuration**: Configuring Arvak compilation
8. **Moments**: Cirq's parallel operation structure
9. **Parametrized Circuits**: Variational algorithms
10. **Native Gates**: Hardware-native gate sets
11. **Export for Production**: Saving circuits for CLI execution

## Key Advantages of Cirq + Arvak

- **Hardware-Native + Optimized**: Cirq's gate sets with Arvak's compilation
- **2D Layouts**: GridQubit support for superconducting processors
- **Variational Algorithms**: Parametrized circuits with Arvak backends
- **Moments**: Fine-grained control over parallelism

## Next Steps

- Explore **01_core_arvak.ipynb** for pure Arvak programming
- Explore **02_qiskit_integration.ipynb** for Qiskit integration
- Explore **03_qrisp_integration.ipynb** for Qrisp integration
- Use the Arvak CLI for production execution on real hardware
- Check out the Arvak dashboard for interactive development

## Resources

- Arvak Documentation: https://github.com/hiq-lab/HIQ
- Cirq Documentation: https://quantumai.google/cirq
- Cirq GitHub: https://github.com/quantumlib/Cirq
- OpenQASM Spec: https://openqasm.com/